# Chat Completion API

In [ ]:
!pip install openai

## 환경설정
1. openai 사이트에서 API KEY 발급
2. google colab - secret탭 - `OPENAI_API_KEY` 등록

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕 내 이름은 차은우야"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "안녕하세요, 차은우님! 만나서 반가워요 😊  \n오늘은 어떤 도움이 필요하신가요?  \n편하게 궁금한 점이나 하고 싶은 이야기를 말씀해 주세요!"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=7960,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 만나서 반가워요 😊  \n오늘은 어떤 도움이 필요하신가요?  \n편하게 궁금한 점이나 하고 싶은 이야기를 말씀해 주세요!'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력 데이터/예시
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용

In [ ]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요'
title_before = '겁나 빡센 작업으로 끼니 거르기 일쑤인 노동자들의 애환'

# 역할/페르소나/지시사항
system_message = """
기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완곡하고 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

# Steps (작업 절차)
1. 기사 제목을 읽고 주요 내용을 이해하세요.
2. 제목이 전달하고자 하는 메시지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결하게 정리하세요.

# Output Format
기사의 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.
- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

# Examples (2-shot-prompting)
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차 회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차 회사들"

# Extra Instructions
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메시지를 잃어버리면 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 해당 표현을 완화하세요.

"""

user_message = f"""
다음 기사 제목을 짧게 교정해 주세요.

{title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message.content)

- 원래 제목: "테이의 FM 개꿀 라디오 방송에 주목해주세요"
- 교정 제목: "테이의 FM 라디오 방송에 주목하세요"


In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):

  # 역할/페르소나/지시사항
  system_message = """
  기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조 등에 있어서 교정작업을 수행해 주세요.

  - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
  - 독자의 관심을 끌 수 있도록 간결하면서 임팩트 있는 표현을 사용하세요.
  - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완곡하고 중립적인 어조로 수정하세요.
  - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고 의미를 적절히 유지하도록 제목을 교정하세요.

  # Steps (작업 절차)
  1. 기사 제목을 읽고 주요 내용을 이해하세요.
  2. 제목이 전달하고자 하는 메시지를 명확하게 반영하는지 검토하세요.
  3. 맞춤법, 문법, 의미 전달의 정확성 등을 점검하고 적절히 수정하세요.
  4. 제목이 자연스럽고 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결하게 정리하세요.

  # Output Format
  기사의 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.
  - 원래 제목: [기사 원래 제목]
  - 교정 제목: [교정 기사 제목]

  # Examples (2-shot-prompting)
  - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
  - 교정 제목: "서울 대형 화재, 수백명 대피"

  - 원래 제목: "전기자동차 판매량 급감에 내연차 회사들이 즐거워하는 중입니다."
  - 교정 제목: "전기차 판매량 급감에 웃는 내연차 회사들"

  # Extra Instructions
  - 제목이 너무 길면, 간결하게 줄이되, 핵심 메시지를 잃어버리면 안됩니다.
  - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
  - 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 해당 표현을 완화하세요.

  """

  user_message = f"""
  다음 기사 제목을 짧게 교정해 주세요.

  {title_before}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: "주말 미친 폭우 예상, 모두들 무사하시길~"
- 교정 제목: "주말 폭우 예상, 안전 유의하세요"


## 연애코치 ReAct

In [3]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):

  # 역할/페르소나/지시사항
  system_message = """
  << System Instruction>>
  어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로서 사용자의 고민을 해결해 주세요.

  << Output Format >>
  1. 상황분석:
  2. 행동계획:
  3. 실행:

  """

  user_message = f"""
  사용자 현재상황:
  {prompt}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

In [4]:
prompt = '저 이번주말에 여자친구와 100일인데 어떡하죠?'
output = dating_coach(prompt)
print(output)

1. 상황분석: 
   - 현재 여자친구와의 관계가 100일을 맞이하는 중요한 순간입니다. 이는 서로의 사랑을 확인하고 더욱 깊어질 수 있는 기회입니다. 그러나, 특별한 날인 만큼 당신은 긴장하거나 어떤 특별한 방식으로 이 날을 기념하고 싶을 것입니다. 상대방이 기쁘고 행복할 수 있도록 준비하는 것이 중요합니다.

2. 행동계획: 
   - 우선, 여자친구의 취향과 관심사를 고려하여 작은 선물이나 이벤트를 준비하세요. 예를 들어, 그녀가 좋아하는 꽃이나 작은 액세서리, 편지 같은 개인적인 터치를 추가한 선물이 좋습니다.
   - 특별한 날에는 함께 할 수 있는 특별한 활동을 계획하세요. 로맨틱한 저녁식사, 야외 피크닉, 영화 관람 등의 활동이 있습니다. 그녀가 좋아하는 장소나 활동을 고려해 결정하세요.
   - 그와 관련하여 감정적인 메시지를 전하는 것도 좋습니다. 서로의 관계에 대한 생각이나 감사의 마음을 표현하는 시간을 가지고, 서로의 사랑을 다시 한번 확인하는 시간을 가져보세요.

3. 실행: 
   - 오늘부터 그녀의 취향에 맞는 선물을 선택하고, 필요한 경우 미리 예약해야 하는 활동 (레스토랑 예약 등)을 진행하세요.
   - 100일 기념일에 전달할 편지나 메시지를 미리 작성하여 감정이 잘 전달되도록 준비하세요.
   - 기념일 당일에는 준비한 내용을 충실히 수행하면서, 사랑하는 그녀가 특별하고 소중하게 느낄 수 있도록 모든 순간을 소중히 여기세요. 그녀에게 “100일을 함께할 수 있어 정말 행복하다”는 식의 대화를 통해 그 순간을 더욱 특별하게 만들어주세요.


## 냉털마스터 ReAct
- 사용자는 냉장고에 남아있는 음식재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- Reasonging/Action을 끌어낼 수 있는 적절한 프롬프팅을 작성하세요.

In [23]:
def fridge_raid_master(prompt, model='gpt-4o-mini', temperature=1, top_p=1):

  # 역할/페르소나/지시사항
  system_message = """
  << System Instruction>>
  다이어트 중인 사용자에게 냉장고에 있는 재료를 활용한 최고의 메뉴를 추천하고 요리하는 방법을 자세하게 제공하세요.

  # 상세 사항

  - 냉장고 내부의 일반적인 재료를 중심으로 창의적이고 실용적인 요리 방법을 작성하세요.
  - 한국 문화와 관련된 요리 스타일이나 전통적인 요리법을 반영하세요.
  - 3개 이상의 요리법을 제안하여 사용자가 다양한 선택지를 가질 수 있도록 합니다.
  - 요리 방법은 손쉽게 따라할 수 있게 단계별로 설명합니다.
  - 다이어트 식단을 우선적으로 추천하고, 메뉴를 추천하는 이유로 어필하세요.
  - 요리를 위해 추가적으로 필요한 재료가 있다면, 일반 가정에서 쉽게 구할 수 있는 재료 위주로 결정하세요.
  - 준비 시간이 15분 이내인 요리만 추천하세요.

  # 출력 형식

  - 각 요리는 [메뉴명], [재료], [준비 시간], [요리 단계], [추천 이유]의 형식으로 구성합니다.
  - 요리 단계는 번호로 매겨 명확하게 설명합니다.

  # 예시

  **예시 1:**

  _입력_: 밥, 김치, 계란

  _출력_:
  - [메뉴명]: 김치볶음밥
  - [재료]: 밥 1공기, 김치 100g, 계란 2개
  - [준비 시간]: 약 15분
  - [요리 단계]:
    1. 기름을 두른 팬에 김치를 볶습니다.
    2. 밥을 넣고 함께 볶습니다.
    3. 계란을 풀어 넣고 섞습니다.
    4. 간이 맞으면 접시에 담아냅니다.
  - [추천 이유]

  (참고: 실제 요리는 다양한 재료 조합과 맛을 살려야 합니다.)

  # 주의 사항

  - 재료의 종류에 따라 알레르기 정보를 제공할 수도 있습니다.
  - 요리법은 일반 가정에서 쉽게 구할 수 있는 재료를 기준으로 작성하세요.
  - 준비 시간이 15분 이내인 요리만 추천하세요.
  - 반드시 사용자가 입력한 모든 재료를 사용해야 하는 것은 아닙니다.

  """

  user_message = f"""
  냉털 재료 :
  {prompt}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

In [26]:
from IPython.display import Markdown

prompt = '김치, 두부, 계란, 파'
output = fridge_raid_master(prompt)

display(Markdown((output)))

- [메뉴명]: 두부 김치 전
- [재료]: 김치 100g, 두부 200g, 계란 2개, 파 약간, 소금, 후추 약간
- [준비 시간]: 약 15분
- [요리 단계]:
  1. 두부를 물기를 제거한 후 잘게 부숴줍니다.
  2. 김치를 잘게 썰고, 파는 송송 썰어줍니다.
  3. 큰 볼에 두부, 김치, 계란, 파, 소금, 후추를 넣고 잘 섞어줍니다.
  4. 팬에 기름을 두르고, 혼합물을 동그랗게 올려 중약불에서 앞뒤로 노릇하게 구워줍니다. 약 3~4분 정도씩 구워줍니다.
  5. 구워진 두부 김치 전을 접시에 담고, 원하는 대로 플레이팅합니다.
- [추천 이유]: 이 요리는 저칼로리이며 단백질이 풍부한 두부와 발효식품인 김치의 조화로 건강에 좋은 영양소가 가득합니다. 또한, 간단하게 조리할 수 있어 다이어트를 하면서도 맛있고 영양가 있는 한 끼를 즐길 수 있도록 도와줍니다.

In [28]:
prompt = '김치, 두부, 계란, 파'
output = fridge_raid_master(prompt)

print(output)

- [메뉴명]: 김치 두부 스크램블
- [재료]: 김치 100g, 두부 150g, 계란 2개, 파 1대, 소금, 후추 약간
- [준비 시간]: 약 15분
- [요리 단계]:
    1. 두부를 손으로 잘 부숴서 큰 덩어리가 없도록 합니다.
    2. 팬에 기름을 두르고 중불에서 다진 파를 볶아 향을 내줍니다.
    3. 파가 투명해지면, 부순 두부를 넣고 함께 볶습니다.
    4. 두부가 약간 노릇해지면, 잘게 썬 김치를 넣고 볶습니다.
    5. 재료가 잘 섞이면, 계란을 풀어 넣고 재빨리 저어가며 스크램블해줍니다.
    6. 간이 맞으면 소금과 후추로 간을 추가하고, 약간 익힌 후 접시에 담아 마무리합니다.
- [추천 이유]: 김치 두부 스크램블은 단백질이 풍부한 두부와 계란, 그리고 발효된 김치의 식이섬유가 조화를 이루어 다이어트에 적합합니다. 또한, 간단하고 빠르게 만들 수 있어 바쁜 일상에서도 부담 없이 즐길 수 있는 건강식입니다. 파의 향이 더해져 풍미가 살아납니다.


## 면접질문 생성 JSON 출력

In [34]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):

  # 역할/페르소나/지시사항
  system_message = """
  << System Instruction>>
  당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로서, 해당 분야의 으뜸가는 면접관입니다.
  매번 그룹사의 인재를 발굴하기 위해 면접 질문과 모범 답안을 작성하고 있습니다.

  <<지시사항>>
  - 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접 질문과 답변을 작성해 주세요.
  - 하드스킬과 소프트스킬 두 가지 섹션으로 나누어 작성해 주세요.
  - 각 스킬별로 질문/답변을 3개씩 만들어 주세요.

  <<츨력형식>>
  - 출력은 json형식으로만 반환되어야 합니다.

    {{
      "hard_skill": [
        {{
          "question": "",
          "answer": ""
        }}
      ],
      "soft_skill": [
        {{
          "question": "",
          "answer": ""
        }}
      ]
    }}

  """

  user_message = f"""
  채용 공고 :
  {job_posting}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      }
    ],
    response_format={
      "type": "json_object"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

In [36]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
output = job_interview(job_posting)

print(type(output)) # str
print(output)

<class 'str'>
{
  "hard_skill": [
    {
      "question": "Python 프로그래밍 관련 경험을 설명해 주시겠습니까?",
      "answer": "저는 Python을 사용하여 다양한 머신러닝 프로젝트에서 데이터 전처리, 모델 학습 및 배포에 참여한 경험이 있습니다. 특히, Pandas와 NumPy를 활용하여 데이터를 다루고, Scikit-learn을 이용해 다양한 알고리즘의 모델링을 진행한 적이 있습니다. 또한, Flask 프레임워크를 사용하여 API를 구현하고, 이를 통해 모델을 서빙한 경험도 있습니다."
    },
    {
      "question": "Docker와 Kubernetes를 사용한 컨테이너 오케스트레이션 경험에 대해 설명해 주세요.",
      "answer": "저는 Docker를 사용하여 애플리케이션의 컨테이너 이미지를 만들고, Kubernetes를 통해 이를 클러스터 환경에서 관리한 경험이 있습니다. 이를 통해 CI/CD 파이프라인을 구축하고, Jenkins와 통합하여 자동 배포를 구현한 경험이 있습니다. 이러한 경험은 애플리케이션의 지속적인 통합과 배포를 효율적으로 관리하는 데 큰 도움이 되었습니다."
    },
    {
      "question": "MLflow와 같은 머신러닝 플랫폼을 사용한 경험이 있나요?",
      "answer": "네, MLflow를 사용하여 모델 실험 추적과 관리, 그리고 배포 프로세스를 자동화한 경험이 있습니다. MLflow의 Tracking 기능을 통해 여러 실험의 메트릭을 비교하고, 모델의 성능을 기반으로 최적화를 진행했습니다. 이 과정을 통해 모델 개발 주기를 단축하고, 협팀 간의 협업을 원활하게 할 수 있었습니다."
    }
  ],
  "soft_skill": [
    {
      "question": "팀원과의 협업에서 중요한 요소는 무엇이라고 생각하시나요?",
      "answer": "저는 팀원과의 신뢰와 효과적인

In [39]:
import json

# 응답 데이터가 json형식이 아니면 오류 발생!
data = json.loads(output)
print(type(data)) # dict
print(data)

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
{'hard_skill': [{'question': 'Python 프로그래밍 관련 경험을 설명해 주시겠습니까?', 'answer': '저는 Python을 사용하여 다양한 머신러닝 프로젝트에서 데이터 전처리, 모델 학습 및 배포에 참여한 경험이 있습니다. 특히, Pandas와 NumPy를 활용하여 데이터를 다루고, Scikit-learn을 이용해 다양한 알고리즘의 모델링을 진행한 적이 있습니다. 또한, Flask 프레임워크를 사용하여 API를 구현하고, 이를 통해 모델을 서빙한 경험도 있습니다.'}, {'question': 'Docker와 Kubernetes를 사용한 컨테이너 오케스트레이션 경험에 대해 설명해 주세요.', 'answer': '저는 Docker를 사용하여 애플리케이션의 컨테이너 이미지를 만들고, Kubernetes를 통해 이를 클러스터 환경에서 관리한 경험이 있습니다. 이를 통해 CI/CD 파이프라인을 구축하고, Jenkins와 통합하여 자동 배포를 구현한 경험이 있습니다. 이러한 경험은 애플리케이션의 지속적인 통합과 배포를 효율적으로 관리하는 데 큰 도움이 되었습니다.'}, {'question': 'MLflow와 같은 머신러닝 플랫폼을 사용한 경험이 있나요?', 'answer': '네, MLflow를 사용하여 모델 실험 추적과 관리, 그리고 배포 프로세스를 자동화한 경험이 있습니다. MLflow의 Tracking 기능을 통해 여러 실험의 메트릭을 비교하고, 모델의 성능을 기반으로 최적화를 진행했습니다. 이 과정을 통해 모델 개발 주기를 단축하고, 협팀 간의 협업을 원활하게 할 수 있었습니다.'}], 'soft_skill': [{'question': '팀원과의 협업에서 중요한 요소는 무엇이라고 생각하시나요?', 'answer': '저는 팀원과의 신뢰와 효과적인 커뮤니케이션이 협업에서 가장 중요한 요소라고 생각합니다. 프로젝트 진행 중에는 지속적인 피드백과 아이디어 공유를 통해 각 팀원의 의견을 반영하는 것이 중요하며, 이